In [ ]:
# global packages
import tensorflow as tf
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import sys
sys.path.append('../')
from src.argparse import argparse
from src.region_to_bbox import region_to_bbox
from src.pprint_params import pprint_params
from src.Tracker import Tracker

In [ ]:
hp = {"z_lr":0,"scale_step":1.02}
evaluation = {"video": "vot2016_ball1"}
run = {"visualization":1}
hp,evaluation,run,env,design = argparse(hp, evaluation, run)
pprint_params((hp,evaluation,run,env,design),['hyperparams','evaluation','run','environment','design'])

In [ ]:
import csv
video_folder = os.path.join(env.root_dataset, evaluation.dataset, evaluation.video) 
gt_file = os.path.join(video_folder, 'groundtruth.txt')
gt = np.genfromtxt(gt_file, delimiter=',')
# bbox is in format <cx,cy,w,h>
init_bbox = region_to_bbox(gt[evaluation.start_frame])
frame_name_list = [f for f in os.listdir(video_folder) if f.endswith(".jpg")]
frame_name_list.sort()
assert len(gt) == len(frame_name_list), ('Number of frames and number of GT lines should be equal.')
# create an instance of the class Tracker and initialize it with groundtruth
first_frame = plt.imread(os.path.join(video_folder, frame_name_list[evaluation.start_frame]))
tracker = Tracker(init_bbox, first_frame, design)

In [ ]:
# initialize figure and visualize
if run.visualization:    
    fig, ax = plt.subplots(1)
    r = patches.Rectangle(
        (tracker.bbox.cx-tracker.bbox.w/2, tracker.bbox.cy-tracker.bbox.h/2),
        tracker.bbox.w, tracker.bbox.h,
        linewidth=2, edgecolor='r', fill=False)
    ax.imshow(tracker.frame)
    ax.add_patch(r)
    plt.show()

In [ ]:
# used to pad the crops
avg_chan = np.mean(tracker.frame, axis=(0,1))

In [ ]:
# fixed multiplicative factors to scale up/down the target
scale_factors = hp.scale_step**np.linspace(-np.ceil(hp.scale_num/2), np.ceil(hp.scale_num/2), hp.scale_num)